# OAS-hepB Data Overview

In [ ]:
# Retina quality plots
%config InlineBackend.figure_format = 'retina'

## Load data

In [ ]:
import os
import gzip
import json
import pandas as pd

METADATA_DIR = 'data/meta'
UNITS_LISTS_DIR = METADATA_DIR + '/units-list'
SEQ_DIR = 'data/seq'
UNITS_LIST_FILE_EXT = '.txt'
DATAUNIT_FILE_EXT = '.parquet'
METADATA_FILE_EXT = '.parquet'

def load_data(study):
    """
    Loads data from given study.
    
    Args:
        study    name of a study to be loaded.

    Returns:
        dataframe of sequences and metadata from given study and dataframe
        consisting of dataunits metadata from given study.
    """
    
    dataunits_list_path = f'{UNITS_LISTS_DIR}/{study}{UNITS_LIST_FILE_EXT}'

    with open(dataunits_list_path) as dataunits_list_file:
        dataunits = [line.strip() for line in dataunits_list_file.readlines()]

    ddfs = []
    meta_dfs = []
    for dataunit in dataunits:
        data_path = f'{SEQ_DIR}/{study}/{dataunit}{DATAUNIT_FILE_EXT}'
        df = pd.read_parquet(data_path)
        
        metadata_path = f'{METADATA_DIR}/{study}/{dataunit}{METADATA_FILE_EXT}'
        meta_df = pd.read_parquet(metadata_path)
        
        for col in meta_df.columns:
            df[col] = meta_df[col][0]

        ddfs.append(df)
        meta_dfs.append(meta_df)
    
    data = pd.concat(ddfs, axis=0)
    metadata = pd.concat(meta_dfs, axis=0)

    return data, metadata

In [ ]:
STUDIES = ['Galson_2015a', 'Galson_2016']

In [ ]:
data_dfs, metadata_dfs = map(list, list(zip(*[load_data(study) for study in STUDIES])))

data = pd.concat(data_dfs)
metadata = pd.concat(metadata_dfs)

In [ ]:
data.info()
data.head()

In [ ]:
metadata.info()
metadata.head()

## Data overview

### Size

In [ ]:
def human_readable_number(number):
    return f'{number:,}'

def size_overview(df):
    print(f'Total number of sequences: {df.shape[0]:,}')
    unique_seq_cnt = df['seq'].nunique()
    print(f'Number of unique sequences: {unique_seq_cnt:,}')
    unique_cdr3_cnt = df['cdr3'].nunique()
    print(f'Number of unique CDR3 sequences: {unique_cdr3_cnt:,}')

    seq_cnt_col_name = 'Total sequences'
    unique_seq_cnt_col_name = 'Unique sequences'
    unique_cdr3_cnt_col_name = 'Unique CDR3 sequences'

    df_visit_grpby = df.groupby('Longitudinal')
    visit_sizes_df = df_visit_grpby.size().reset_index(name=seq_cnt_col_name).set_index('Longitudinal')
    visit_sizes_df[unique_seq_cnt_col_name] = df_visit_grpby['seq'].nunique()
    visit_sizes_df[unique_cdr3_cnt_col_name] = df_visit_grpby['cdr3'].nunique()

    formatters = {
        seq_cnt_col_name: human_readable_number,
        unique_seq_cnt_col_name: human_readable_number,
        unique_cdr3_cnt_col_name: human_readable_number
    }
    display(visit_sizes_df.style.format(formatters))

In [ ]:
size_overview(data)

### Sequence counts from subjects

In [ ]:
from bin.plotting import barplot

ax = barplot(data['Subject'].value_counts(), title='Number of sequences from subjects');
ax.set(xlabel='Number of sequences');

### Sequences lengths

In [ ]:
import seaborn as sns
import numpy as np

def lengths_plot(data, target, title):
    lengths = data[target].str.len()

    ax = sns.distplot(lengths, bins=np.arange(lengths.min(), lengths.max() + 1), kde_kws={'bw':1})
    ax.set_title(title)
    ax.set(xlabel='Length')

In [ ]:
lengths_plot(data, 'seq', 'Sequence lengths')

In [ ]:
from matplotlib import pyplot as plt

def for_each_length_plots(df, for_each_col, target, title):
    for_each_values = df[for_each_col].unique()

    for for_each in for_each_values:
        lengths_plot(df[df[for_each_col] == for_each],  target, title.format(for_each))
        plt.show()

In [ ]:
for_each_length_plots(data, 'BType', 'seq', '{} - sequence lengths')

In [ ]:
for_each_length_plots(data, 'Isotype', 'seq', '{} - sequence lengths')

In [ ]:
lengths_plot(data, 'cdr3', 'CDR3 lengths')

In [ ]:
for_each_length_plots(data, 'BType', 'cdr3', '{} - CDR3 lengths')

In [ ]:
for_each_length_plots(data, 'Isotype', 'cdr3', '{} - CDR3 lengths')

### B-cell labels

In [ ]:
ax = barplot(metadata.groupby('BType')['Size'].sum(), title='B-cell labels - sequence counts');
ax.set(ylabel='B-cell label', xlabel='Number of sequences');

In [ ]:
def count_plots(data, for_each_col, target, groupby_col):
    for_each_unique_values = sorted(data[for_each_col].unique())

    for for_each_val in for_each_unique_values:
        data_to_plot = data[data[for_each_col] == for_each_val].groupby(groupby_col)[target].sum()
        ax = barplot(data_to_plot, title=f'{for_each_val} - {groupby_col} counts');
        ax.set(xlabel='Number of sequences');
        plt.show()

In [ ]:
count_plots(metadata, 'Longitudinal', 'Size', 'BType')

### Subjects visits

In [ ]:
def subject_visits_overview(df):
    df_groupby_subject = df.groupby('Subject')
    
    display(df_groupby_subject['Longitudinal'].unique())
    ax = barplot(df_groupby_subject['Longitudinal'].nunique(), title='Number of visits for subjects')
    ax.set(xlabel='Number of visits');

In [ ]:
subject_visits_overview(data)

### hepB antibodies count

In [ ]:
def subject_hepb_counts(df):
    df_groupby_subject = df.groupby('Subject')
    HEBP_TYPE = 'HepB+B-cells'
    ncols = 3
    nrows = int(np.ceil(df_groupby_subject.ngroups / ncols))
    
    _, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(6 * ncols, 4 * nrows), sharey=True)

    for (key, ax) in zip(df_groupby_subject.groups.keys(), axes.flatten()):
        subject = df_groupby_subject.get_group(key)

        visit_values = sorted(subject['Longitudinal'].unique())
        hepb_cnts = [len(subject[(subject['Longitudinal'] == visit) & (subject['BType'] == HEBP_TYPE)]) for visit in visit_values]

        sns.pointplot(visit_values, hepb_cnts, ax=ax)
        ax.set(title=key, ylabel='hepB ABs (symlog)', yscale='symlog')

    plt.tight_layout()
    plt.show()

In [ ]:
subject_hepb_counts(data)